# Script to populate database - 2nd part;

## Insert CSV

This script get generate CSV in 1st part and insert in database

## Set external variables

In [ ]:
# server of database;
server = ""

# name of database;
database = ""

# port of database;
port = 1433

# user of database;
user = ""

# password of password;
password = ""

# csv with 1st part to insert in database;
csv_file = ""

# flag to clear or not table before insert csv data;
clear_table = True

# Max register insert per time
max_insert = 2000

## Script

### Librarys

In [ ]:
import pymssql
import pandas as pd
import time

### Functions

In [ ]:
def sql_query(sql_query, table_columns):
    try:
        cursor = connection.cursor()
        cursor.execute(sql_query)
        row = cursor.fetchone()
        response = []
        while row:
            response.append(row)
            row = cursor.fetchone()
        return response
    except Exception as fail:
        raise Exception('exception get sql, fail: {}'.format(fail))

In [ ]:
def sql_execute(sql_script):
    try:
        cursor = connection.cursor()
        cursor.execute(sql_script)
        connection.commit()
    except Exception as fail:
        connection.rollback()
        raise Exception('exception update sql {}, fail: {}'.format(sql_script, fail), True)

In [ ]:
def sql_execut_emany(sql_script, list_exec, n_errors=0):
    try:
        cursor = connection.cursor()
        cursor.executemany(sql_script, list_exec)
        connection.commit()
    except Exception as fail:
        if n_errors < 5:
            print('exception insert fail: {}; Try again soon!'.format(fail))
            time.sleep(10)
            sql_execut_emany(sql_script, list_exec, n_errors+1)
        else:
            connection.rollback()
            raise Exception('exception update sql {}, fail: {}'.format(
                sql_script, fail), True)

In [ ]:
def sql_insert(sql_insert, list_inserts):
    n = int(len(list_inserts) / max_insert) + 1

    for i in range(n):
        list_ins_part = list_inserts[max_insert*i: max_insert*(i+1)]
        sql_execut_emany(sql_insert, list_ins_part)

In [ ]:
def insert_information(informations):
    if not len(informations) > 0:
        exit

    sql = '''
    INSERT INTO EMISSAO_DFE
    (EMDF_SISTEMA, EMDF_MODELO, EMDF_DATA_EMISSAO, EMDF_NUMERO, EMDF_SERIE,
    EMDF_CHAVE, EMDF_CNPJ, EMDF_RAZAO_SOCIAL, EMDF_FANTASIA, EMDF_ENDERECO,
    EMDF_BAIRRO, EMDF_MUNICIPIO)
    VALUES(%s, %s, CONVERT(DATETIME, %s, 103), %d, %s, %s, %s, %s, %s, %s, %s, %s)
    '''

    list_inserts = []
    for inf in informations:
        new_insert = [value for tag, value in dict(inf).items()]
        list_inserts.append(tuple(new_insert))

    sql_insert(sql, list_inserts)

In [ ]:
connection = pymssql.connect(host=server, port=port, database=database, user=user, password=password)

### Script

In [ ]:
df = pd.read_csv(csv_file)
df.head()

In [ ]:
df.tail()

In [ ]:
print('N rows original: {}'.format(df.shape[0]))

df_duplicate = df[df.duplicated()]
print('N rows duplicate: {}'.format(df_duplicate.shape[0]))

df = df.drop_duplicates()
print('N rows after drop duplicates: {}'.format(df.shape[0]))

In [ ]:
query = '''
SELECT *
  FROM EMISSAO_DFE
'''

# query = 'drop table EMISSAO_DFE'
# sql_execute(query)

# query = '''
# CREATE TABLE EMISSAO_DFE(
#     EMDF_ID int identity(1,1) not null,
#     EMDF_SISTEMA varchar(10) null,
#     EMDF_MODELO  varchar(5) null,
#     EMDF_DATA_EMISSAO datetime null,
#     EMDF_NUMERO int null,
#     EMDF_SERIE varchar(3) null,
#     EMDF_CHAVE varchar(44) null,
#     EMDF_CNPJ varchar(14) null,
#     EMDF_RAZAO_SOCIAL varchar(60) null,
#     EMDF_FANTASIA varchar(60) null,
#     EMDF_ENDERECO varchar(200) null,
#     EMDF_BAIRRO varchar(60) null,
#     EMDF_MUNICIPIO varchar(60) null
# PRIMARY KEY CLUSTERED (
#     EMDF_ID ASC
# )
# WITH (
#     PAD_INDEX = OFF,
#     STATISTICS_NORECOMPUTE = OFF,
#     IGNORE_DUP_KEY = OFF,
#     ALLOW_ROW_LOCKS = ON,
#     ALLOW_PAGE_LOCKS = ON,
#     FILLFACTOR = 90,
#     OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
# ) ON [PRIMARY]
# '''
# sql_execute(query)



r_query = sql_query(query, '')
print(len(r_query))

if (len(r_query) > 0) and clear_table:
    query = 'DELETE FROM EMISSAO_DFE'
    sql_execute(query)
    query = 'DBCC CHECKIDENT (EMISSAO_DFE, RESEED, 0)'
    sql_execute(query)

In [ ]:
n_gestNFe, n_StacNFe, n_StacCTe = 0, 0, 0
informations = []
data_list = []
check_list = []
for index, value in df.iterrows():
    sistema = value.sistema if not pd.isnull(value.sistema) else ''
    modelo = value.tipo_nota if not pd.isnull(value.tipo_nota) else ''
    data = value.data if not pd.isnull(value.data) else ''
    numero = value.nf_nfc_ct if not pd.isnull(value.nf_nfc_ct) else ''
    serie = value.serie if not pd.isnull(value.serie) else ''
    chave = value.chave if not pd.isnull(value.chave) else ''
    cnpj = value.CNPJ if not pd.isnull(value.CNPJ) else ''
    razao_social = value.razao_social if not pd.isnull(value.razao_social) else ''
    fantasia = value.nome_fantasia if not pd.isnull(value.nome_fantasia) else ''
    endereco = value.endereco if not pd.isnull(value.endereco) else ''
    bairro = value.bairro if not pd.isnull(value.bairro) else ''
    cidade = value.cidade if not pd.isnull(value.cidade) else ''

    data_list.append(data)
    check_list.append(value.mes_ano)
    
    n_StacNFe += 1 if sistema == 'StacNFe' else 0
    n_StacCTe += 1 if sistema == 'StacCTe' else 0
    n_gestNFe += 1 if sistema == 'GestNFe' else 0
    
    # Tmp aqui, mover para construcao do csv insercao
    
    if not pd.isnull(data):
        day, month, year = data.split('/')
        if len(year) < 4:
            year = '19' if int(year) >= 30 else '20' + year;
            
        if int(day) > 31 or int(month) > 12:
            aux = day
            day = month
            month = aux

        day = '0' + day if len(day) < 2 else day
        month = '0' + month if len(month) < 2 else month

        data = '/'.join([day, month, year])

    
    informations.append({"EMDF_SISTEMA": sistema,
                         "EMDF_MODELO": modelo,
                         "EMDF_DATA_EMISSAO": data,
                         "EMDF_NUMERO": numero,
                         "EMDF_SERIE": serie,
                         "EMDF_CHAVE": chave,
                         "EMDF_CNPJ": cnpj,
                         "EMDF_RAZAO_SOCIAL": razao_social,
                         "EMDF_FANTASIA": fantasia,
                         "EMDF_ENDERECO": endereco,
                         "EMDF_BAIRRO": bairro,
                         "EMDF_MUNICIPIO": cidade})

In [ ]:
print('N register of StacNFe: {}'.format(n_StacNFe))
print('N register of StacCTe: {}'.format(n_StacCTe))
print('N register of gestNFe: {}'.format(n_gestNFe))
print('Total register: {}'.format(n_StacNFe + n_StacCTe + n_gestNFe))

In [ ]:
insert_information(informations)